In [1]:
import numpy as np 
import pandas as pd
import json 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from copy import deepcopy

import Utils
import models

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('PMT_only_children.xlsx', sheet_name='teens')

In [3]:
df

,Unnamed: 0,age,code,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,...,N_ADD3,N_DIV1,N_DIV2,N_DIV3,N_MUL1,N_MUL2,N_MUL3,N_SUB1,N_SUB2,N_SUB3
0,1.0,17,P-DPO-17-F-BP-HSEL-MOON2021-ANXX-2112,0.916667,0.714286,0.947368,0.909091,0.461538,1.000000,0.476190,...,3,4,4,3,6,4,2,7,4,3
1,1.0,17,P-VSU-17-F-BI-AKGY-SAON2021-EIXX-2166,0.909091,0.916667,0.968254,0.700000,0.750000,1.000000,0.812500,...,5,31,4,3,29,4,6,35,7,3
2,1.0,16,P-DDA-16-M-AY-1253-MOON2021-ANXX-2113,0.526316,0.518519,1.000000,0.454545,0.125000,0.982759,0.421053,...,11,51,19,6,63,22,8,53,24,6
3,1.0,14,P-EPR-14-F-8P-1538-MOON2021-SSXX-2114,1.000000,0.600000,0.944444,1.000000,1.000000,0.950000,1.000000,...,3,21,2,2,21,1,2,19,2,2
4,1.0,17,P-ASU-17-F-B?-0171-MOON2021-AIXX-2115,1.000000,1.000000,1.000000,0.918367,1.000000,1.000000,0.888889,...,16,55,10,5,60,21,9,56,23,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,1.0,13,P-VMU-13-F-7A-0006-VOON2022-VSXX-2448,0.944444,0.941176,0.666667,0.794118,0.454545,0.285714,0.897436,...,8,27,5,4,35,3,4,32,10,5
163,NaN,14,P-PMO-14-M-8A-RZ25-VION2022-EBXX-2457,0.956522,0.904762,0.500000,0.961538,1.000000,0.818182,0.973684,...,20,25,11,9,37,13,11,36,17,9
164,NaN,17,P-SKU-17-F-11B-1210-MOON2022-SGXX-2458,0.857143,0.545455,0.444444,0.897436,0.300000,0.500000,1.000000,...,4,35,3,3,41,9,1,31,3,4
165,NaN,14,P-MMI-14-M-7C-1874-MOON2022-SGXX-2460,0.972973,0.923077,0.800000,0.930233,0.900000,0.800000,0.972222,...,8,40,9,4,35,7,4,39,12,5


In [4]:
df.columns

Index(['Unnamed: 0', 'age', 'code', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3',
       'ACC_DIV1', 'ACC_DIV2', 'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3',
       'ACC_SUB1', 'ACC_SUB2', 'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3',
       'RT_DIV1', 'RT_DIV2', 'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3',
       'RT_SUB1', 'RT_SUB2', 'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1',
       'N_DIV2', 'N_DIV3', 'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2',
       'N_SUB3'],
      dtype='object')

In [5]:


df = df.drop(columns=['code', 'Unnamed: 0'])

seed = 0xAB0BA
np.random.seed(seed)

In [6]:
df['Accuracy ALL level1'] = df['ACC_ADD1'] + df['ACC_SUB1'] + df['ACC_MUL1'] + df['ACC_DIV1']
df['Accuracy ALL level2'] = df['ACC_ADD2'] + df['ACC_SUB2'] + df['ACC_MUL2'] + df['ACC_DIV2']
df['Accuracy ALL level3'] = df['ACC_ADD3'] + df['ACC_SUB3'] + df['ACC_MUL3'] + df['ACC_DIV3']
df['Accuracy ALL ADD'] = df['ACC_ADD1'] + df['ACC_ADD2'] + df['ACC_ADD3']
df['Accuracy ALL SUB'] = df['ACC_SUB1'] + df['ACC_SUB2'] + df['ACC_SUB3']
df['Accuracy ALL MUL'] = df['ACC_MUL1'] + df['ACC_MUL2'] + df['ACC_MUL3']
df['Accuracy ALL DIV'] = df['ACC_DIV1'] + df['ACC_DIV2'] + df['ACC_DIV3']


In [7]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,N_SUB1,N_SUB2,N_SUB3,Accuracy ALL level1,Accuracy ALL level2,Accuracy ALL level3,Accuracy ALL ADD,Accuracy ALL SUB,Accuracy ALL MUL,Accuracy ALL DIV
0,17,0.916667,0.714286,0.947368,0.909091,0.461538,1.000000,0.476190,0.276596,0.982759,...,7,4,3,3.061948,2.202420,4.450395,2.578321,3.030268,1.735545,2.370629
1,17,0.909091,0.916667,0.968254,0.700000,0.750000,1.000000,0.812500,0.571429,0.936170,...,35,7,3,3.221591,3.015873,4.248701,2.794012,2.922054,2.320099,2.450000
2,16,0.526316,0.518519,1.000000,0.454545,0.125000,0.982759,0.421053,0.272727,0.936170,...,53,24,6,1.901914,1.133637,4.405318,2.044834,2.203780,1.629950,1.562304
3,14,1.000000,0.600000,0.944444,1.000000,1.000000,0.950000,1.000000,1.000000,1.000000,...,19,2,2,3.875000,3.600000,3.894444,2.544444,2.875000,3.000000,2.950000
4,17,1.000000,1.000000,1.000000,0.918367,1.000000,1.000000,0.888889,0.888889,1.000000,...,56,23,9,3.807256,3.388889,3.101000,3.000000,1.601000,2.777778,2.918367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,13,0.944444,0.941176,0.666667,0.794118,0.454545,0.285714,0.897436,0.300000,0.666667,...,32,10,5,3.524887,2.410008,2.244048,2.552288,2.228175,1.864103,1.534377
163,14,0.956522,0.904762,0.500000,0.961538,1.000000,0.818182,0.973684,0.866667,1.000000,...,36,17,9,3.891744,3.715873,3.318182,2.361284,2.944444,2.840351,2.779720
164,17,0.857143,0.545455,0.444444,0.897436,0.300000,0.500000,1.000000,0.818182,0.200000,...,31,3,4,3.615690,1.963636,1.715873,1.847042,1.732540,2.018182,1.697436
165,14,0.972973,0.923077,0.800000,0.930233,0.900000,0.800000,0.972222,0.875000,1.000000,...,39,12,5,3.826647,3.555220,3.314286,2.696050,2.522648,2.847222,2.630233


In [8]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3',
       'Accuracy ALL level1', 'Accuracy ALL level2', 'Accuracy ALL level3',
       'Accuracy ALL ADD', 'Accuracy ALL SUB', 'Accuracy ALL MUL',
       'Accuracy ALL DIV'],
      dtype='object')

In [9]:
df['RT ALL level1'] = df['RT_ADD1'] + df['RT_SUB1'] + df['RT_MUL1'] + df['RT_DIV1']
df['RT ALL level2'] = df['RT_ADD2'] + df['RT_SUB2'] + df['RT_MUL2'] + df['RT_DIV2']
df['RT ALL level3'] = df['RT_ADD3'] + df['RT_SUB3'] + df['RT_MUL3'] + df['RT_DIV3']
df['RT ALL ADD'] = df['RT_ADD1'] + df['RT_ADD2'] + df['RT_ADD3']
df['RT ALL SUB'] = df['RT_SUB1'] + df['RT_SUB2'] + df['RT_SUB3']
df['RT ALL MUL'] = df['RT_MUL1'] + df['RT_MUL2'] + df['RT_MUL3']
df['RT ALL DIV'] = df['RT_DIV1'] + df['RT_DIV2'] + df['RT_DIV3']


In [10]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,Accuracy ALL SUB,Accuracy ALL MUL,Accuracy ALL DIV,RT ALL level1,RT ALL level2,RT ALL level3,RT ALL ADD,RT ALL SUB,RT ALL MUL,RT ALL DIV
0,17,0.916667,0.714286,0.947368,0.909091,0.461538,1.000000,0.476190,0.276596,0.982759,...,3.030268,1.735545,2.370629,18.339609,18.165353,6.383327,11.769206,11.941363,7.164502,12.013217
1,17,0.909091,0.916667,0.968254,0.700000,0.750000,1.000000,0.812500,0.571429,0.936170,...,2.922054,2.320099,2.450000,23.068129,42.327670,14.620346,12.011963,24.703630,17.202691,26.097860
2,16,0.526316,0.518519,1.000000,0.454545,0.125000,0.982759,0.421053,0.272727,0.936170,...,2.203780,1.629950,1.562304,11.547892,9.810222,13.644003,6.179476,14.111742,8.052421,6.658480
3,14,1.000000,0.600000,0.944444,1.000000,1.000000,0.950000,1.000000,1.000000,1.000000,...,2.875000,3.000000,2.950000,49.222846,70.711121,37.341409,28.709197,48.930877,42.530490,37.104811
4,17,1.000000,1.000000,1.000000,0.918367,1.000000,1.000000,0.888889,0.888889,1.000000,...,1.601000,2.777778,2.918367,72389.378995,44890.757754,22.753149,44965.601603,27439.598776,10.510160,44887.179359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,13,0.944444,0.941176,0.666667,0.794118,0.454545,0.285714,0.897436,0.300000,0.666667,...,2.228175,1.864103,1.534377,9.060570,24.909429,33.343012,13.471615,17.694199,21.129013,15.018186
163,14,0.956522,0.904762,0.500000,0.961538,1.000000,0.818182,0.973684,0.866667,1.000000,...,2.944444,2.840351,2.779720,9.855264,21.468104,26.993316,8.086761,16.377222,15.713007,18.139693
164,17,0.857143,0.545455,0.444444,0.897436,0.300000,0.500000,1.000000,0.818182,0.200000,...,1.732540,2.018182,1.697436,9.057335,30.588500,48.252327,18.641106,22.975077,23.418526,22.863454
165,14,0.972973,0.923077,0.800000,0.930233,0.900000,0.800000,0.972222,0.875000,1.000000,...,2.522648,2.847222,2.630233,8.806887,28.163674,47.259308,16.341945,19.560122,26.641862,21.685940


In [11]:
# Calculating new columns for aggregated NofCorr
df['NofCorr ALL level1'] = df['N_ADD1'] + df['N_SUB1'] + df['N_MUL1'] + df['N_DIV1']
df['NofCorr ALL level2'] = df['N_ADD2'] + df['N_SUB2'] + df['N_MUL2'] + df['N_DIV2']
df['NofCorr ALL level3'] = df['N_ADD3'] + df['N_SUB3'] + df['N_MUL3'] + df['N_DIV3']
df['NofCorr ALL ADD'] = df['N_ADD1'] + df['N_ADD2'] + df['N_ADD3']
df['NofCorr ALL SUB'] = df['N_SUB1'] + df['N_SUB2'] + df['N_SUB3']
df['NofCorr ALL MUL'] = df['N_MUL1'] + df['N_MUL2'] + df['N_MUL3']
df['NofCorr ALL DIV'] = df['N_DIV1'] + df['N_DIV2'] + df['N_DIV3']  # Corrected from RT_DIV3 to N_DIV3


In [12]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3',
       'Accuracy ALL level1', 'Accuracy ALL level2', 'Accuracy ALL level3',
       'Accuracy ALL ADD', 'Accuracy ALL SUB', 'Accuracy ALL MUL',
       'Accuracy ALL DIV', 'RT ALL level1', 'RT ALL level2', 'RT ALL level3',
       'RT ALL ADD', 'RT ALL SUB', 'RT ALL MUL', 'RT ALL DIV',
       'NofCorr ALL level1', 'NofCorr ALL level2', 'NofCorr ALL level3',
       'NofCorr ALL ADD', 'NofCorr ALL SUB', 'NofCorr ALL MUL',
       'NofCorr ALL DIV'],
      dtype='object')

In [13]:
def construct_dataset(df, target_column):
    # Determine if the target is aggregated
    is_aggregated_target = 'ALL' in target_column


    
    # Initialize lists to hold columns to include and exclude
    columns_to_exclude = []
    columns_to_include = []
    
    # Base columns categories
    accuracy_columns = [col for col in df.columns if col.startswith('ACC_')]
    reaction_time_columns = [col for col in df.columns if col.startswith('RT_')]
    n_of_corr_columns = [col for col in df.columns if col.startswith('N_') and 'ofCorr' not in col]
    aggregated_columns = [col for col in df.columns if 'ALL' in col]

    relevant = []
    agg_name = ''
    metric = ''
    if ('ACC' in target_column) or ('Accuracy' in target_column):
        metric = 'ACC'
        relevant = accuracy_columns
        agg_name = 'Accuracy'
    elif 'RT' in target_column:
        metric = 'RT'
        relevant = reaction_time_columns
        agg_name = 'RT'
    elif ('N_' in target_column) or ('ofCorr' in target_column):
        metric = 'N' 
        relevant = n_of_corr_columns
        agg_name = 'NofCorr'


    # Exclude components for an aggregated target
    if is_aggregated_target:
        # Identifying the components based on the target
        if 'ADD' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'ADD' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])
        elif 'SUB' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'SUB' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'MUL' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'MUL' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'DIV' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'DIV' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'level' in target_column:
            level = target_column.split()[-1][-1]  # Extracting level part
            columns_to_exclude.extend([col for col in relevant if level in col])

            # remove aggregates by operation, otherwise could calculate
            operations = ['ADD', 'SUB', 'MUL', 'DIV']
            columns_to_exclude.extend([f'{agg_name} ALL {operation}' for operation in operations])
    else:
        # For non-aggregated targets, exclude related aggregated columns
        level = target_column.split('_')[-1][-1]
        operation = target_column.split('_')[-1][:-1]
        related_aggregates = [f'{agg_name} ALL {operation}'] + [f'{agg_name} ALL level{level}']
        columns_to_exclude.extend(related_aggregates)

        #remove N with same operation and same level for ACC
        if metric == 'ACC':
            columns_to_exclude.extend([f'N_{operation}{level}'])
    
    # Aggregated columns to always exclude if they are not the target
   # columns_to_exclude.extend([col for col in aggregated_columns if col != target_column])
   # print(columns_to_exclude)
    # Include columns that are not in the exclude list and not the target column
    columns_to_include = [col for col in df.columns if col not in columns_to_exclude and col != target_column]

    # Preparing the dataset
    X = df[columns_to_include]
    y = df[target_column]
    
    return X, y


In [168]:
X, y = construct_dataset(df, 'RT ALL level1')

['RT_ADD1', 'RT_DIV1', 'RT_MUL1', 'RT_SUB1', 'RT ALL ADD', 'RT ALL SUB', 'RT ALL MUL', 'RT ALL DIV']


In [140]:
X.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV2', 'RT_DIV3',
       'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2', 'RT_SUB3',
       'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3', 'N_MUL1',
       'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3', 'Accuracy ALL level1',
       'Accuracy ALL level2', 'Accuracy ALL level3', 'Accuracy ALL ADD',
       'Accuracy ALL SUB', 'Accuracy ALL MUL', 'Accuracy ALL DIV',
       'RT ALL level2', 'RT ALL level3', 'RT ALL ADD', 'RT ALL SUB',
       'RT ALL MUL', 'NofCorr ALL level1', 'NofCorr ALL level2',
       'NofCorr ALL level3', 'NofCorr ALL ADD', 'NofCorr ALL SUB',
       'NofCorr ALL MUL', 'NofCorr ALL DIV'],
      dtype='object')

In [21]:
results = {}

for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)
    
    res = models.LinRegStatmodels(X, y)
    
    r2 = res.rsquared
    params = res.params.index
    
    results[name] = res

100%|██████████| 58/58 [00:16<00:00,  3.49it/s]


In [22]:
results['RT_DIV2'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                RT_DIV2   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          6.691e+06
Date:                Sat, 03 Feb 2024   Prob (F-statistic):                        0.00
Time:                        21:28:33   Log-Likelihood:                         -546.35
No. Observations:                 167   AIC:                                      1121.
Df Residuals:                     153   BIC:                                      1164.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
ACC_ADD3           -7.5947      2.927     -2.594      0.010     -13.378      -1.812
ACC_MUL1          -16.9534      2.706     -6.266      0.000     -22.299     -11.608
ACC_MUL3            7.4236      2.198      3.377      0.001       3.081      11.766
RT_ADD1             0.7404      0.034     21.818      0.000       0.673       0.807
RT_ADD2            -0.7306      0.043    -16.979      0.000      -0.816      -0.646
RT_ADD3             0.7204      0.120      6.006      0.000       0.483       0.957
RT_DIV1             0.4723      0.009     52.033      0.000       0.454       0.490
RT_DIV3            -0.7188      0.119     -6.020      0.000      -0.955      -0.483
RT_MUL1            -2.0397      0.009   -224.958      0.000      -2.058      -2.022
RT_MUL2             2.4917      0.177     14.063      0.000       2.142       2.842
RT_SUB1             0.3544      0.007     52.274      0.000       0.341       0.368
RT_SUB2            -0.1179      0.002    -51.862      0.000      -0.122      -0.113
RT_SUB3            -0.1182      0.002    -52.769      0.000      -0.123      -0.114
RT ALL level1      -0.4726      0.009    -52.294      0.000      -0.490      -0.455
RT ALL ADD          0.7303      0.043     16.988      0.000       0.645       0.815
RT ALL SUB          0.1182      0.002     52.118      0.000       0.114       0.123
NofCorr ALL ADD     0.1335      0.030      4.426      0.000       0.074       0.193
==============================================================================
Omnibus:                        4.414   Durbin-Watson:                   1.864
Prob(Omnibus):                  0.110   Jarque-Bera (JB):                5.235
Skew:                          -0.141   Prob(JB):                       0.0730
Kurtosis:                       3.820   Cond. No.                     5.23e+16
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 3.13e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [171]:
params_frequency = {}
for name in df.columns:
    params_frequency[name] = set()

params_frequency['const'] = set()

for res in results:
    for param_name in results[res].params.index:
        params_frequency[param_name].add(res)


In [174]:

for name in params_frequency:
    new_input = str(params_frequency[name])
    if new_input == 'set()':
        new_input = ''
    params_frequency[name] = new_input

with open("results/params_in_models_new.json", "w") as outfile:
    json.dump(params_frequency, outfile, indent=4)


r2_dict = {}
for name in results:
    r2_dict[name] = results[name].rsquared

with open("results/r2_LinReg_new.json", "w") as outfile:
    json.dump(r2_dict, outfile, indent=4)


rmse_dict = {}
for name in results:
    rmse_dict[name] = float(np.sqrt(results[name].mse_total))

with open("results/rmse_LinReg_new.json", "w") as outfile:
    json.dump(rmse_dict, outfile, indent=4)

In [14]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier



In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


In [16]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.XGBReg(X, y)

    result[name] = (op, rmse, r2, model)

100%|██████████| 58/58 [06:10<00:00,  6.39s/it]


In [17]:
result['ACC_ADD1']

((100, 2, 0.1, 0.7),
 0.05692233073906272,
 0.5174619389423523,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.1, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [17]:
rmse_dic_xgb = {}
r2_dic_xgb = {}
opt_params = {}
for elem in result:
    r2_dic_xgb[elem] = result[elem][2]
    rmse_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]
    

In [20]:
with open("results/rmse_XGB_new.json", "w") as outfile:
    json.dump(rmse_dic_xgb, outfile, indent=4)
with open("results/params_XGB_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)
with open("results/r2_XGB_new.json", "w") as outfile:
    json.dump(r2_dic_xgb, outfile, indent=4)

RF

In [16]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.RFReg(X, y)

    result[name] = (op, rmse, r2, model)

100%|██████████| 58/58 [12:06<00:00, 12.53s/it]


In [18]:
rmse_dic_rf = {}
opt_params = {}
r2_dic_rf = {}

for elem in result:
    r2_dic_rf[elem] = result[elem][2]
    rmse_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [19]:
with open("results/rmse_RF_new.json", "w") as outfile:
    json.dump(rmse_dic_rf, outfile, indent=4)

with open("results/params_RF_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

with open("results/r2_RF_new.json", "w") as outfile:
    json.dump(r2_dic_rf, outfile, indent=4)

comparison


In [36]:
with open('rmse_RF_new.json') as file:
    res_rf = json.load(file)
with open('params_RF_new.json') as file:
    params_rf = json.load(file)

with open('rmse_XGB_new.json') as file:
    res_gb = json.load(file)

with open('params_XGB_new.json') as file:
    params_gb = json.load(file)

In [38]:
res_rf

{'1': 0.27264622347353584, '2': 0.4232025548675165, '3': 0.6024865819197409}

In [39]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff > 0:
        opt_params = params_gb[elem]
        model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])
    else:
        opt_params = params_rf[elem]
        model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X,y = df_list[elem]
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBRegressor):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        X,y = df_list[elem]
        FI = list(zip(opt_models[elem].feature_importances_, X.columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

100%|██████████| 3/3 [00:02<00:00,  1.40it/s]


In [41]:
with open('FeatureImportance5_new.json', 'w') as file:
     json.dump(FI5, file, indent=4)